In [1]:
import os

GLOVE_DIR = './glove.6B/'
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 400


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# data = pd.read_csv('../input/spam.csv',encoding='latin-1')
data = pd.read_csv('./spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics

num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(texts,tags, test_size = 0.3)
mat_texts_tr = tok.texts_to_matrix(x_train,mode='count')
mat_texts_tst = tok.texts_to_matrix(x_test,mode='count')

max_len = 400
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)

word_index = tok.word_index

cnn_texts_mat = sequence.pad_sequences(x_train,maxlen=max_len)
max_len = 100
cnn_texts_mat_tst = sequence.pad_sequences(x_test,maxlen=max_len)


Using TensorFlow backend.


In [4]:
# dictionary of just words were using
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    if word in word_index.keys():
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 6518 word vectors.


In [21]:
# dict of whole whole glove
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [5]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [6]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [7]:
def check_model(model,xtr,ytr,xts,yts):
    model.fit(xtr,ytr,batch_size=32,epochs=10,verbose=1,validation_split=0.3)
    print(' ')
    model.evaluate(xts,yts)

In [8]:
from keras.layers import LSTM
def get_LSTM_v4():    
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])

    print('Train...')
    return model

In [9]:
m = get_LSTM_v4()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst ,y_test)

Train...
Train on 2730 samples, validate on 1170 samples
Epoch 1/10
2730/2730 [==============================] - 102s 37ms/step - loss: 0.3071 - acc: 0.8938 - binary_accuracy: 0.8938 - val_loss: 0.1806 - val_acc: 0.9538 - val_binary_accuracy: 0.9538
Epoch 2/10
2730/2730 [==============================] - 105s 38ms/step - loss: 3.8065 - acc: 0.9355 - binary_accuracy: 0.9355 - val_loss: 0.1755 - val_acc: 0.9282 - val_binary_accuracy: 0.9282
Epoch 3/10
2730/2730 [==============================] - 104s 38ms/step - loss: nan - acc: 0.2755 - binary_accuracy: 0.2755 - val_loss: nan - val_acc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00
Epoch 4/10
2730/2730 [==============================] - 103s 38ms/step - loss: nan - acc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00
Epoch 5/10
2730/2730 [==============================] - 105s 38ms/step - loss: nan - acc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - val_loss: nan - val_acc: 0

KeyboardInterrupt: 